大問2. 乳がんデータセット
================

## Q1. 実装
つぎの条件にしたがって、乳がん(breast_cancer)の分類を行ってください。

なお、指定された条件以外にも、必要だと思われる処理等を思いついた場合に自由に追加してもらって構いません。

### 条件
- 使用するデータ: [breast_cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html)
- 評価指標: 正答率(Accuracy)
- 評価プロトコル: ホールドアウト(交差検証は不要)
- アルゴリズム: サポートベクターマシン(Support Vector Machine)

In [0]:
# データの準備
from sklearn.datasets import load_breast_cancer

breast_cancer_dataset = load_breast_cancer()

In [0]:
# データセットの分割
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(breast_cancer_dataset.data,
                                                   breast_cancer_dataset.target,
                                                   random_state=24)


In [3]:
# 学習
from sklearn.svm import SVC

svm = SVC(C=1.0)
svm.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

## Q2. 評価その1
クライアントに説明することを想定して、最終的な評価や分析結果も記述してください。


In [4]:
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

1.0
0.6293706293706294


訓練データのスコアはよいが、テストデータのスコアが62%と悪い。
改良の必要がある。

## Q3. モデルの改良
クライアントから「正答率は最低でも90%以上は欲しい」という要望がありました。

正答率が90%以上になるようにモデルを改良してください。このとき、Q1の条件は自由に変更してもらって構いません。

なお、Q1の時点で既に90%以上の正答率を出している場合には、Q3およびQ4の回答は不要です。


In [5]:
# 桁数がバラバラなのでスケーリングする
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# 学習データに対してMinとMaxを計算
# あくまで X_train のみ(y_train)は不要！
scaler.fit(X_train) 

MinMaxScaler(copy=True, feature_range=(0, 1))

In [0]:
# 学習データをスケーリング
X_train_scaled = scaler.transform(X_train)

In [0]:
# テストデータをスケーリング
X_test_scaled = scaler.transform(X_test)

In [12]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=0.1)
logreg.fit(X_train_scaled, y_train)

# 交差検証
from sklearn.model_selection import cross_val_score
import numpy as np
cv_score_logreg = cross_val_score(LogisticRegression(),
                                  X_train_scaled,
                                  y_train,
                                  cv=10)


ロジスティック回帰: 0.9579734219269103


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

## Q4. 評価その2
あらためて、クライアントに説明することを想定して、最終的な評価や分析結果も記述してください。

In [24]:
cv_score_logreg = cross_val_score(LogisticRegression(),
                                  X_test_scaled,
                                  y_test,
                                  cv=10)

print(f"ロジスティック回帰: {np.mean(cv_score_logreg)}")

ロジスティック回帰: 0.9299999999999999


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

データの大小にばらつきがあったため、スケーリングで目盛りを合わせた。
その結果最終的なスコアは92%となった。

## Q5. 混同行列による評価
今回のモデルにおける混同行列を求め、その結果についての説明をしてください。


In [25]:
from sklearn.metrics import confusion_matrix

pred_logreg = logreg.predict(X_test)
confusion_matrix(y_test, pred_logreg)

array([[53,  0],
       [90,  0]])

偽陽性がないので、良性と判断したけど悪性だったということはなかった。